In [1]:
import tarfile

In [2]:
tf = tarfile.open("dogsCats.tar.xz")
tf.extractall()

In [3]:
!ls

README.md            dogsCats.tar.xz      test1.tar.xz
Recitation 1.ipynb   sampleSubmission.csv train.tar.xz


In [4]:
tf = tarfile.open("test1.tar.xz")
tf.extractall()

In [5]:
tf = tarfile.open("train.tar.xz")
tf.extractall()

In [6]:
!ls

README.md            sampleSubmission.csv train
Recitation 1.ipynb   test1                train.tar.xz
dogsCats.tar.xz      test1.tar.xz


In [7]:
# We have two folders

train_dir = "./train"
test_dir = "./test1"

In [9]:
# more import statements 
from keras.preprocessing.image import ImageDataGenerator 

Using TensorFlow backend.
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/anaconda3/lib/python3.7/site-packages/tensorflow/py

In [23]:
 """
all images will be rescaled by 1./255
We want the same size for each image 
Need to put this in GPU 
"""
# if you want to do data augmentation, you would augment within the ImageDataGenerator() parameters. 
train_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)
#This is saying I want 20% of my data to be split 
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size=20, class_mode='binary', subset='training')



Found 20000 images belonging to 2 classes.


In [31]:
 """
all images will be rescaled by 1./255
We want the same size for each image 
Need to put this in GPU 
"""

validation_generator = train_datagen.flow_from_directory(train_dir, target_size = (150,150), batch_size=20, class_mode='binary', subset='validation')





Found 5000 images belonging to 2 classes.


In [25]:
for data_batch, labels_batch in train_generator: 
    print("data batch shape: ", data_batch.shape)
    print("label batch shape: ", labels_batch.shape)
    break

data batch shape:  (20, 150, 150, 3)
label batch shape:  (20,)


In [26]:
from keras import layers 
from keras import models 

model = models.Sequential()

# 3x3 filter
# 32 is the number of filters 
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Flatten())# reshape function 
model.add(layers.Dropout(0.5)) #ratio that you want to drop at every pass, suppose your accuracy 
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
#output is one because binary classification class 

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)      

In [27]:
"""
train the cnn 
specify optimization method, learning rate, loss function, and metric 
"""

from keras import optimizers 
model.compile(loss="binary_crossentropy", optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [29]:
history = model.fit_generator (train_generator, epochs=1, validation_data = validation_generator, validation_steps=10)

Epoch 1/1
 105/1000 [==>...........................] - ETA: 9:53 - loss: 0.6420 - acc: 0.6338

KeyboardInterrupt: 

In [30]:
# we are using the generator to optimize ram so we call .fit_generator()

Plot accuracy against epochs

more training - more epochs 
training accuracy was 99% 
overfitting

why? 
3M parameters and jhust 2K training samples 

Brute force solution - get more data 

Trick 1: drop out 
Trick 2: data augmentation

1 epochs is one pass 
Drop out only before the 1st dense layer to regularlize the 1st dense layer

because the the 1st dense layer has too many trainiable parameters 